In [1]:
from os.path import dirname, join, pardir
from cobra.io import read_sbml_model
from escher import Builder
from dfba import DfbaModel, ExchangeFlux, KineticVariable

data_dir = "/Users/phivri/Documents/Github/Biosustain/MoGeoModel"

In [2]:
# DfbaModel instance initialized with cobra model
fba_model = read_sbml_model(join(data_dir, "Models/iJR904.xml"))
fba_model.solver = "glpk"

#Desired ratio of max growth, chosen from PPP:
ratio = 0.5

XFlux = fba_model.slim_optimize()*ratio

with fba_model:
    fba_model.reactions.get_by_id("BIOMASS_Ecoli").bounds = XFlux, XFlux
    fba_model.objective = "EX_ac_e"
    PFlux = fba_model.slim_optimize()

set_flux = fba_model.problem.Constraint(
    XFlux * fba_model.reactions.EX_ac_e.flux_expression - PFlux * fba_model.reactions.BIOMASS_Ecoli.flux_expression,
    lb=0,
    ub=0)
fba_model.add_cons_vars(set_flux)

solution = fba_model.optimize()

dfba_model = DfbaModel(fba_model)

Builder(model = fba_model, map_name='e_coli_core.Core metabolism', reaction_data = solution.fluxes.to_dict())

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


Builder(reaction_data={'ALATA_L2': 0.0, 'ALAabc': 0.0, 'ALAt2r': 0.0, 'ALCD19': 0.0, 'ALDD19xr': 0.0, 'ALDD2x'…

In [3]:
fba_model.medium

{'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_co2_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_na1_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 20.0,
 'EX_pi_e': 999999.0,
 'EX_so4_e': 999999.0}

In [4]:
"""Anaerobic growth of *E. coli* on glucose and xylose.

Organism -> _Escherichia coli str. K-12 substr. MG1655_
Model in http://bigg.ucsd.edu/models/iJR904
"""

# instances of KineticVariable (default initial conditions are 0.0, but can be
# set here if wanted e.g. Oxygen)
X = KineticVariable("Biomass")
Gluc = KineticVariable("Glucose")
Oxy = KineticVariable("Oxygen", initial_condition=0.24)
Eth = KineticVariable("Ethanol")
Actt = KineticVariable("Acetate")

# add kinetic variables to dfba_model
dfba_model.add_kinetic_variables([X, Gluc, Oxy, Eth, Actt])

# instances of ExchangeFlux
mu = ExchangeFlux("BIOMASS_Ecoli")
v_G = ExchangeFlux("EX_glc__D_e")
v_O = ExchangeFlux("EX_o2_e")
v_E = ExchangeFlux("EX_etoh_e")
v_A = ExchangeFlux("EX_ac_e")

# add exchange fluxes to dfba_model
dfba_model.add_exchange_fluxes([mu, v_G, v_O, v_E, v_A])

# add rhs expressions for kinetic variables in dfba_model
dfba_model.add_rhs_expression("Biomass", mu * X)
dfba_model.add_rhs_expression("Glucose", v_G * 180.1559 * X / 1000.0)
dfba_model.add_rhs_expression("Oxygen", v_O * 16.0 * X / 1000.0)
dfba_model.add_rhs_expression("Ethanol", v_E * 46.06844 * X / 1000.0)
dfba_model.add_rhs_expression("Acetate", v_E * 46.06844 * X / 1000.0)

# add lower/upper bound expressions for exchange fluxes in dfba_model together
# with expression that must be non-negative for correct evaluation of bounds
dfba_model.add_exchange_flux_lb("EX_glc__D_e", 10.5 * (Gluc / (0.0027 + Gluc)) * (1 / (1 + Eth / 20.0)), Gluc)
dfba_model.add_exchange_flux_lb("EX_o2_e", 15.0 * (Oxy / (0.024 + Oxy)), Oxy)

# add initial conditions for kinetic variables in dfba_model biomass (gDW/L),
# metabolites (g/L)
dfba_model.add_initial_conditions(
    {
        "Biomass": 0.03,
        "Glucose": 15.5,
        "Oxygen": 0.0,
        "Ethanol": 0.0,
        "Acetate": 0.0
    }
)

# simulate model across interval t = [0.0,25.0](hours) with outputs for plotting
# every 0.1h and optional list of fluxes
concentrations, trajectories = dfba_model.simulate(
    0.0, 25.0, 0.1, ["BIOMASS_Ecoli","EX_glc__D_e", "EX_etoh_e", "EX_ac_e"]
)

# generate plots of results (in this case using plotlly)


In [5]:
trajectories

,time,BIOMASS_Ecoli,EX_glc__D_e,EX_etoh_e,EX_ac_e
0,0.0,0.000000,0.000000,0.000000,0.000000
1,0.1,0.229247,-10.497480,9.405887,7.720169
2,0.2,0.229225,-10.496774,9.405480,7.719445
3,0.3,0.229203,-10.496050,9.405064,7.718705
4,0.4,0.229181,-10.495310,9.404638,7.717947
...,...,...,...,...,...
185,18.5,0.182717,-8.966699,8.525157,6.153210
186,18.6,0.181760,-8.935206,8.507037,6.120972
187,18.7,0.180647,-8.898616,8.485985,6.083517
188,18.8,0.179053,-8.846177,8.455815,6.029839
